In [1]:
import numpy as np
import os, sys
import math
from scipy import ndimage, misc
from astropy.coordinates import Angle
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy import linalg
import csv
import pickle
from matplotlib.colors import LogNorm
from matplotlib import colors
import configparser
import json
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import logging
%matplotlib inline
power = 3
power_dir = "output_neid_order_trace"
# power_dir = "output_paras_order_trace"

In [2]:
from modules.order_trace.src.alg import OrderTraceAlg
from kpfpipe.models.level0 import KPF0
load_dotenv()
KPF_TESTDATA = os.getenv('KPFPIPE_TEST_DATA') + '/'
print(KPF_TESTDATA)
MODULE_DIR = '../../modules/order_trace/'

/Users/cwang/documents/KPF/ownCloud/KPF-Pipeline-TestData/


In [ ]:
from tests.regression import test_order_trace
# add __init__.py tentatively for testing the following functions.
test_order_trace.test_init_exceptions()

In [ ]:

test_order_trace.test_locate_clusters_paras()
test_order_trace.test_form_clusters_paras()
test_order_trace.test_advanced_cluster_cleaning_paras()
test_order_trace.test_clean_clusters_on_borders_paras()
test_order_trace.test_merge_clusters_and_clean_paras()
test_order_trace.test_find_widths_paras()


In [ ]:
# input: spectral fits is from dropbox KPF-Pipeline-TestData/order_trace_test
spectral_fits=  KPF_TESTDATA + 'polygon_clipping_test/paras_data/paras.flatA.fits'
output_dir = MODULE_DIR + 'results/result_data'
output_img_dir = MODULE_DIR + 'results/result_img'

# output
cluster_info_xy_csv = output_dir +'/cluster_xy_info_paras.csv'

cluster_clean_fits = output_img_dir+'/cluster_form_paras.fits'
cluster_info_clean_csv = output_dir + '/cluster_form_info_paras.csv'

cluster_after_removal_fits = output_img_dir + '/cluster_clean_paras.fits'
cluster_info_after_removal_csv = output_dir + '/cluster_clean_info_paras.csv'

cluster_border_fits = output_img_dir +'/cluster_clean_border_paras.fits'
cluster_info_border_csv = output_dir + '/cluster_clean_border_info_paras.csv'

cluster_merge_fitting = output_img_dir+'/cluster_merge_paras.fits'
cluster_info_merge_fitting_csv = output_dir+'/cluster_merge_info_paras.csv'

result_csv = output_dir + '/cluster_curve_paras.csv'

In [ ]:
# input: spectral fits is from dropbox KPF-Pipeline-TestData/NEIData/FLAT
# simulated_nobias = '../../test_results/kpf/nobias_tester_sim.fits'
# simulated_final = '../../test_results/kpf/finalbias_tester_sim.fits'
spectral_fits= KPF_TESTDATA + '/NEIDdata/FLAT/stacked_2fiber_flat.fits'
output_dir = MODULE_DIR + 'results/result_data'
output_img_dir = MODULE_DIR + 'results/result_img'
png = 'neid_flat_'

# output
cluster_info_xy_csv = output_dir +'/cluster_xy_info_neid.csv'

cluster_clean_fits = output_img_dir+'/cluster_form_neid.fits'
cluster_info_clean_csv = output_dir + '/cluster_form_info_neid.csv'

cluster_after_removal_fits = output_img_dir + '/cluster_clean_neid.fits'
cluster_info_after_removal_csv = output_dir + '/cluster_clean_info_neid.csv'

cluster_border_fits = output_img_dir +'/cluster_clean_border_neid.fits'
cluster_info_border_csv = output_dir + '/cluster_clean_border_info_neid.csv'

cluster_merge_fitting = output_img_dir+'/cluster_merge_neid.fits'
cluster_info_merge_fitting_csv = output_dir+'/cluster_merge_info_neid.csv'

result_csv = output_dir + '/cluster_curve_neid.csv'

In [3]:
# spectral_fits= KPF_TESTDATA + '/KPF-Pipeline-TestData/KPF_Simulated_Data/flat/' + 
#                'KPF_rays-1.0E+05_orders- 71-103_cal-incandescent_sci-incandescent_sky-incandescent_normalized_159_Red.fits'
# spectral_fits = KPF_TESTDATA + 'KPF_Simulated_Data/test_biasflats/kpf_green_red_flat.fits'
# spectral_fits = '/Users/cwang/documents/KPF/KPF-Pipeline/test_data/kpf/220316/assembled_KPF_220316_153200_L0.fits'
# spectral_fits = '/Users/cwang/documents/KPF/KPF-Pipeline/test_data/kpf/220411/lev0/KPF_220411_193700_L0.fits'
# spectral_fits = '/Users/cwang/documents/KPF/KPF-Pipeline/test_data/kpf/L0/20220505/'
#spectral_fits = '/Users/cwang/documents/KPF/KPF-Pipeline/test_data/kpf/ait/red/20220413/L0/KP.20220413.22310.00_2D_bkgsub.fits'
#spectral_fits = '/Users/cwang/documents/KPF/KPF-Pipeline/test_data/kpf/ait/green/20220415/L0/KP.20220415.23260.00_2D_bkgsub.fits'
#spectral_fits = '/Users/cwang/documents/KPF/KPF-Pipeline/test_data/kpf/2D/20220510/KP.20220510.04445.31_2D.fits'
#spectral_fits = '/Users/cwang/documents/KPF/KPF-Pipeline/test_data/kpf/2D/20220516_log/KP.20220516.54179.83.fits'
spectral_fits = '/Users/cwang/documents/KPF/KPF-Pipeline/test_data/kpf/2D/20220518/KP.20220518.17215.78.fits'
print(spectral_fits)
# spectral_fits = '/Users/cwang/documents/KPF/KPF-Pipeline/test_results/kpf/data/kpf_green_red_flat.fits'
output_dir = MODULE_DIR + 'results/result_data'
output_img_dir = MODULE_DIR + 'results/result_img'

# spectral_fits= KPF_TESTDATA + '/KPF-Pipeline-TestData/KPF_Simulated_Data/flat/kpf_green_L0_20210426T000000.fits'
png = 'kpf_flat_'

# output
cluster_info_xy_csv = output_dir +'/cluster_xy_info_kpf.csv'

cluster_clean_fits = output_img_dir+'/cluster_form_kpf.fits'
cluster_info_clean_csv = output_dir + '/cluster_form_info_kpf.csv'

cluster_after_removal_fits = output_img_dir + '/cluster_clean_kpf.fits'
cluster_info_after_removal_csv = output_dir + '/cluster_clean_info_kpf.csv'

cluster_border_fits = output_img_dir +'/cluster_clean_border_kpf.fits'
cluster_info_border_csv = output_dir + '/cluster_clean_border_info_kpf.csv'

cluster_merge_fitting = output_img_dir+'/cluster_merge_kpf.fits'
cluster_info_merge_fitting_csv = output_dir+'/cluster_merge_info_kpf.csv'

result_csv = output_dir + '/cluster_curve_kpf.csv'

/Users/cwang/documents/KPF/KPF-Pipeline/test_data/kpf/2D/20220518/KP.20220518.17215.78.fits


In [4]:
# copy of start_logger from logger.py
def get_level(lvl:str) -> int:
    if lvl == 'debug': return logging.DEBUG
    elif lvl == 'info': return logging.INFO
    elif lvl == 'warning': return logging.WARNING
    elif lvl == 'error': return logging.ERROR
    elif lvl == 'critical': return logging.CRITICAL
    else: return logging.NOTSET

def start_logger(logger_name: str, config: str):
    if config is None: 
        # a config file is not provided, so don't start logger
        print('[{}] missing log configuration...not starting a new logger'.format(
            logger_name))
        return None
    config_obj = configparser.ConfigParser()
    res = config_obj.read(config)
    if res == []:
        return None

    log_cfg = config_obj['LOGGER']

    log_start = log_cfg.get('start_log', False)
    log_path = log_cfg.get('log_path', 'log')
    log_lvl = log_cfg.get('log_level', logging.WARNING)
    log_verbose = log_cfg.getboolean('log_verbose', True)
    # logger.setLevel(get_level(log_lvl))
        
    # if log_start:
    #     # setup a log format
    #     formatter = logging.Formatter('[%(name)s][%(levelname)s]:%(message)s')
    #     # setup a log file
    #     f_handle = logging.FileHandler(log_path, mode='w') # logging to file
    #     f_handle.setLevel(get_level(log_lvl))
    #     f_handle.setFormatter(formatter)
    #     logger.addHandler(f_handle)

    #     if log_verbose: 
    #         # also print to terminal 
    #         s_handle = logging.StreamHandler()
    #         s_handle.setLevel(get_level(log_lvl))
    #         s_handle.setFormatter(formatter)
    #         logger.addHandler(s_handle)
    # return logger


    logger = logging.getLogger(logger_name)
    logger.setLevel(get_level(log_lvl))
    logger.propagate = False

    formatter = logging.Formatter('[%(name)s][%(levelname)s]:%(message)s')
    s_handle = logging.StreamHandler()
    s_handle.setLevel(get_level(log_lvl))
    s_handle.setFormatter(formatter)
    logger.addHandler(s_handle)
    return logger


In [5]:
def plot_imshow(img):
    if (np.nanmax(img) == 1) and (np.nanmin(img) == 0):
        print('is bw image')
        im = plt.imshow(img * -1, cmap='gray')
    else:    # in case there is '0.0' for LogNorm
        im = plt.imshow(img, cmap='gray', interpolation='nearest', norm=LogNorm())
    return im    

In [6]:
# plot image from fits by setting area xmin, xmax, ymin, ymax
def plot_img(img, ymin, ymax, p_w=20, p_h=20, xmin=None, xmax=None, title="", aspect=None, save=''):
    #if is_bw is True:
    #    img = convert_to_bw(img)

    plt.figure(figsize=(p_w, p_h), frameon=False)
    plt.subplot(1, 1, 1)
    if xmin is None:
        xmin = 0
    if xmax is None:
        h, w = np.shape(img)
        xmax = w-1
    s_img = img[:, :]
    im = plot_imshow(s_img)

    #im = plt.imshow(s_img, cmap='gray')
    plt.ylim(ymin, ymax)
    plt.xlim(xmin, xmax)
    plt.title(title)
    if aspect is not None:
        plt.axes().set_aspect(aspect)
    plt.show()
    if save:
        plt.savefig(MODULE_DIR+'results/result_img/'+save+'.png')

In [7]:
def make_fits(data, output_fits):
    hdu = fits.PrimaryHDU(data)
    hdu.writeto(output_fits, overwrite=True)

In [8]:
# make image data in 2D based on selected clusters
def make_2D_data(index, x, y, nx, ny, selected_clusters=None):
    imm = np.zeros((ny, nx), dtype=np.uint8)
    
    if selected_clusters is None:
        selected_clusters = np.arange(1, np.amax(index)+1, dtype=int)

    for idx in selected_clusters:
        crt_idx = np.where(index == idx)[0]
        imm[y[crt_idx], x[crt_idx]] = 1
        
    return imm

In [9]:
# make fits on 2D of all clusters
def make_cluster_fits(index, x, y, nx, ny, fits_path=None):
    imm = make_2D_data(index, x, y,  nx, ny)
    if fits_path is not None:
        make_fits(imm, fits_path)
        ind_max = np.amax(index)
        print('there are '+str(ind_max)+' clusters in total in fits, '+fits_path)
    return imm

In [10]:
# make fits on cluster info (index, x, y)
def make_cluster_info_fits(file_path, nx, ny, x_idx, y_idx, index=None):
    size = x_idx.size
    cluster_data = np.zeros((2, size), dtype=int) if index is None else np.zeros((3, size), dtype=int)
    cluster_data[0, :] = x_idx
    cluster_data[1, :] = y_idx
    if index is not None:
        cluster_data[2, :] = index
    hdu = fits.PrimaryHDU(cluster_data)
    hdu.header['indexmax'] = np.amax(index) if index is not None else 0
    hdu.header['width'] = nx
    hdu.header['height'] = ny
    hdu.writeto(file_path, overwrite=True)
    
def get_cluster_info_fits(file_path: str):
    cluster_info, cluster_info_head = fits.getdata(file_path, header=True)
    x = cluster_info[0, :].astype(int)
    y = cluster_info[1, :].astype(int)
    index_list = None
    if cluster_info_head['indexmax'] != 0:
        index_list = cluster_info[2, :].astype(int)

    return x, y, index_list
    

In [11]:
# make fits on cluster info (index, x, y)
def make_cluster_info_csv(file_path, nx, ny, x_idx, y_idx, index=None):
    size = x_idx.size
    cluster_data = np.zeros((size, 2), dtype=int) if index is None else np.zeros((size, 3), dtype=int)
    cluster_data[:, 0] = x_idx
    cluster_data[:, 1] = y_idx
    if index is not None:
        cluster_data[:, 2] = index
    df = pd.DataFrame(cluster_data)
    df.to_csv(file_path, header=False, index=False)     
    
def get_cluster_info_csv(file_path: str):
    x = None
    y = None
    index = None
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path, header=None, index_col=None)
        row, col = np.shape(df.values)
        x = df.values[:, 0].astype(int) if col >= 1 else None
        y = df.values[:, 1].astype(int) if col >= 2 else None
        index = df.values[:, 2].astype(int) if col >= 3 else None
    return x, y, index

In [12]:
# plot polynomial fitting curve on top of given 2D image
# the cluster orders is settable by order_set
def plot_poly_trace(imm, total_order, coeffs_orders, max_x, max_y, size=20, order_set=None, \
                    title=None, background=False, widths=None, aspect=None, \
                    xmin=None, xmax=None, ymin=None, ymax=None, origin=None, save='', shift=0):
    plt.figure(figsize=(size,size))
    plt.subplot(1, 1, 1)
    im = plot_imshow(imm)
    #im = plt.imshow(imm, cmap='gray', norm=LogNorm())
    
    if order_set is None:
        orders = list(range(1, total_order+1))
    else:
        orders = order_set
        
    x_dist = max_x//20    
    
    o_x, o_y = [origin[0], origin[1]] if origin is not None else [0, 0]
                   
    for o_idx, order in enumerate(orders):
        if (background is not False):
            x_val = np.arange(0, max_x)
            # y value on x range
            y_val = np.polyval(coeffs_orders[order, 0:power+1], x_val) 
            plt.plot(x_val+o_x, y_val+o_y, 'b--')
        #print("x range: ", coeffs_orders[order, power+1], coeffs_orders[order, power+2])    
        # x range
        x_val = np.arange(coeffs_orders[order, power+1], coeffs_orders[order, power+2]+1)
        # y value on x range
        y_val = np.polyval(coeffs_orders[order, 0:power+1], x_val)
        plt.plot(x_val+o_x+shift, y_val+o_y, 'r--')
        
        if widths is not None:
            y_val_bottom = y_val-widths[o_idx][0]
            plt.plot(x_val+o_x, y_val_bottom+o_y, 'g--')
            y_val_top = y_val+widths[o_idx][1]
            plt.plot(x_val+o_x, y_val_top+o_y, 'g--')
        
        # show number of cluster
        s = ((order%15)+1)*x_dist
        if s >= x_val.size:
            dem = int((coeffs_orders[order, power+2] - coeffs_orders[order, power+1])//5)
            s = dem*((order%4)+1)
            #s = x_val.size//2
        plt.text(x_val[s]+o_x, y_val[s]+o_y, str(order), fontsize=12, color='b', fontweight='bold', horizontalalignment='center', verticalalignment='center')
        
    if title is not None:
        plt.title(title, fontsize=12)
    x1 = 0 if xmin is None else xmin
    x2 = max_x if xmax is None else xmax
    y1 = 0 if ymin is None else ymin
    y2 = max_y if ymax is None else ymax
    
    plt.ylim(y1, y2)
    plt.xlim(x1, x2)
    if aspect is not None:
        plt.axes().set_aspect(aspect)
    
    plt.show()
    if save:
        plt.savefig(MODULE_DIR+'results/result_img/'+save+'.png')
    #plt.colorbar(im, fraction=0.046, pad=0.04)

In [13]:
def to_str(afloat):
    new_str = f"{afloat:.4f}"
    return new_str

In [14]:
# json save and load
def save_obj(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

## Usage: Using OrderTarceAlg to extract from the given spectral fits (for NEID or PARAS)

In [ ]:
fits_header = fits.open(spectral_fits)
config_file = MODULE_DIR + 'configs/default_recipe_neid.cfg'
logger = start_logger("OrderTraceAlg", config_file)

width_p = 3
config = configparser.ConfigParser()
config.read(config_file)

width_app = '_width_'+str(width_p) if width_p > 0 else '' 

order_t = OrderTraceAlg(fits_header[0].data, config, logger)

if 'neid' in power_dir:
    csv_file = 'neid_poly_3sigma_gaussian_pixel_' + str(power)+ width_app +'.csv'
else:
    csv_file = 'paras_poly_3sigma_gaussian_pixel_'+ str(power)+ width_app +'.csv'
    
imm_spec, nx, ny = order_t.get_spectral_data()
cluster_info = order_t.extract_order_trace(power_for_width_estimation = width_p, show_time=True, print_debug='')
output_df_to_csv  = result_csv;
df = cluster_info['order_trace_result']
df.to_csv(index=False, path_or_buf=output_df_to_csv, header=False)
#order_t.write_cluster_info_to_csv(cluster_info['widths'], cluster_info['coeffs'], result_poly_width_csv)

In [ ]:
# plot the order result and polynomial fits on top of the identified order
new_x = cluster_info['cluster_x']
new_y = cluster_info['cluster_y']
new_index = cluster_info['cluster_index']
_, nx, ny = order_t.get_spectral_data()

cluster_imm = order_t.make_2d_data(new_index, new_x, new_y)
new_coeffs, errors = order_t.curve_fitting_on_all_clusters(new_index, new_x, new_y)
max_index = np.amax(new_index)

print(np.shape(new_coeffs))
print(max_index)
plot_poly_trace(cluster_imm, max_index, new_coeffs, nx, ny)

## Extracting order trace step by step
### excute the cells from step 1 to step 7 and get visual output for each step

## 1. load spectral file

In [15]:
data_ext = 'GREEN_CCD'
fits_header = fits.open(spectral_fits)
config = configparser.ConfigParser()
config_file = MODULE_DIR + 'configs/default_recipe_kpf.cfg'
config.read(config_file)

lev0_obj = KPF0.from_fits(spectral_fits, data_type='KPF')

flat_data = lev0_obj[data_ext]

#flat_data = fits_header[0].data
#data_range=[2500, -2500, 2500, -2500]
#flat_data = np.where(np.isnan(flat_data), 0.0, flat_data)
png = 'KP.20220518.17215.78_' + data_ext
data_range = None
total_row, total_col = np.shape(flat_data)
# order_t = OrderTraceAlg(flat_data, poly_degree=power, expected_traces=140, config=config)
order_t = OrderTraceAlg(flat_data, poly_degree=power, config=config)
imm_spec, nx, ny = order_t.get_spectral_data()

spe_info = {'data': imm_spec, 'nx': nx, 'ny': ny}

order_t.add_file_logger("") 

# plot_img(imm_spec, 0, ny-1, save=png+'spectral')
power = order_t.get_poly_degree()
print('power: ', power)
print(total_row, total_col, 'row: ', spe_info['ny'], ' column: ', spe_info['nx'])

order_t.set_data_range(data_range)

print(order_t.data_range)

power:  3
4080 4080 row:  4080  column:  4080
[0, 4079, 0, 4079]


## 2. locate_clusters() -- find cluster pixels  and make fits

In [16]:
cluster_xy = order_t.locate_clusters()

yy = np.shape(cluster_xy['cluster_image'])[0]
plot_img(cluster_xy['cluster_image'], 0, yy-1, save=png+'locate_clusters')

OrderTraceAlg: rows_to_reset: None
OrderTraceAlg: cols to reset: None
is bw image


/Users/cwang/Documents/KPF/KPF-Pipeline/venv/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [17]:
make_cluster_info_csv(cluster_info_xy_csv, nx, ny, cluster_xy['x'], cluster_xy['y'])

## 3. form_clusters() or collect_clusters() + remove_cluster_by_size() + reorganize_index()

In [18]:
#cluster_info, dict
#cluster_info = order_t.collect_clusters(list(), list())
cluster_info = order_t.collect_clusters(cluster_xy['x'], cluster_xy['y'])

OrderTraceAlg:  0 OrderTraceAlg:  100 OrderTraceAlg:  200 OrderTraceAlg:  300 OrderTraceAlg:  400 OrderTraceAlg:  500 OrderTraceAlg:  600 OrderTraceAlg:  700 OrderTraceAlg:  800 OrderTraceAlg:  900 OrderTraceAlg:  1000 OrderTraceAlg:  1100 OrderTraceAlg:  1200 OrderTraceAlg:  1300 OrderTraceAlg:  1400 OrderTraceAlg:  1500 OrderTraceAlg:  1600 OrderTraceAlg:  1700 OrderTraceAlg:  1800 OrderTraceAlg:  1900 OrderTraceAlg:  2000 OrderTraceAlg:  2100 OrderTraceAlg:  2200 OrderTraceAlg:  2300 OrderTraceAlg:  2400 OrderTraceAlg:  2500 OrderTraceAlg:  2600 OrderTraceAlg:  2700 OrderTraceAlg:  2800 OrderTraceAlg:  2900 OrderTraceAlg:  3000 OrderTraceAlg:  3100 OrderTraceAlg:  3200 OrderTraceAlg:  3300 OrderTraceAlg:  3400 OrderTraceAlg:  3500 OrderTraceAlg:  3600 OrderTraceAlg:  3700 OrderTraceAlg:  3800 OrderTraceAlg:  3900 OrderTraceAlg:  4000 



In [19]:
# assign index value to cluster_info['index'], where cluster_info['index'] is the same size as cluster_xy['x']
cluster_info = order_t.remove_cluster_by_size(cluster_info, cluster_xy['x'], cluster_xy['y'])

OrderTraceAlg:  there are total  5444146  clusters to test.
OrderTraceAlg: cluster  1  total:  841  w, h 534 4
OrderTraceAlg: cluster  2  total:  36256  w, h 4080 37
OrderTraceAlg: cluster  3  total:  47464  w, h 4080 42
OrderTraceAlg: cluster  4  total:  40917  w, h 4080 40
OrderTraceAlg: cluster  5  total:  22037  w, h 4080 37
OrderTraceAlg: cluster  6  total:  37467  w, h 4080 39
OrderTraceAlg: cluster  7  total:  47523  w, h 4080 41
OrderTraceAlg: cluster  8  total:  41010  w, h 4080 39
OrderTraceAlg: cluster  9  total:  22041  w, h 4080 35
OrderTraceAlg: cluster  10  total:  37544  w, h 4080 38
OrderTraceAlg: cluster  11  total:  47540  w, h 4080 39
OrderTraceAlg: cluster  12  total:  41060  w, h 4080 37
OrderTraceAlg: cluster  13  total:  22018  w, h 4080 34
OrderTraceAlg: cluster  14  total:  37559  w, h 4080 36
OrderTraceAlg: cluster  15  total:  47550  w, h 4080 38
OrderTraceAlg: cluster  16  total:  41088  w, h 4080 37
OrderTraceAlg: cluster  17  total:  22030  w, h 4080 32
O

OrderTraceAlg: cluster  153  total:  1124  w, h 187 29
OrderTraceAlg: cluster  154  total:  677  w, h 80 32
OrderTraceAlg: cluster  155  total:  37961  w, h 4080 29
OrderTraceAlg: cluster  156  total:  48061  w, h 4080 33
OrderTraceAlg: cluster  157  total:  41668  w, h 4080 31
OrderTraceAlg: cluster  158  total:  23867  w, h 4080 26
OrderTraceAlg: cluster  159  total:  388  w, h 66 24
OrderTraceAlg: cluster  160  total:  265  w, h 48 20
OrderTraceAlg: cluster  161  total:  230  w, h 51 19
OrderTraceAlg: cluster  162  total:  301  w, h 53 19
OrderTraceAlg: cluster  163  total:  220  w, h 49 22
OrderTraceAlg: cluster  164  total:  484  w, h 77 25
OrderTraceAlg: cluster  165  total:  533  w, h 97 26
OrderTraceAlg: cluster  166  total:  242  w, h 44 27
OrderTraceAlg: cluster  167  total:  257  w, h 38 25  => remove
OrderTraceAlg: cluster  168  total:  973  w, h 141 28
OrderTraceAlg: cluster  169  total:  216  w, h 39 25  => remove
OrderTraceAlg: cluster  170  total:  658  w, h 94 30
Order

In [20]:
# remove unassigned index
x, y, index_r = order_t.reorganize_index(cluster_info['index'], cluster_xy['x'], cluster_xy['y'])
imm = order_t.make_2d_data(index_r, x, y)   # show image  and make fits and info fits 
plot_img(imm, 0, np.shape(imm)[0]-1, save=png+'first_clean')

is bw image


/Users/cwang/Documents/KPF/KPF-Pipeline/venv/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


### save data from the result of 3 (optional)

In [ ]:
imm=make_cluster_fits(index_r, x, y, nx, ny, cluster_clean_fits)
make_cluster_info_csv(cluster_info_clean_csv, nx, ny, x, y, index_r)
plot_img(imm, 0, np.shape(imm)[0]-1)

### reload clean fits and info fits of step 3. (optional)

In [ ]:
# (optional) reload saved fits 
imm, hdr = fits.getdata(cluster_clean_fits, header=True)
ny, nx = np.shape(imm)
x, y, index_r = get_cluster_info_csv(cluster_info_clean_csv)
plot_img(imm, 0, np.shape(imm)[0]-1)

## 4. advanced_cluster_cleaning_handler() -- cluster cleaning to remove noisy clusters

In [21]:
new_x, new_y, new_index, all_status = order_t.advanced_cluster_cleaning_handler(index_r, x, y)

imm = order_t.make_2d_data(new_index, new_x, new_y)   # show image  and make fits and info fits 
plot_img(imm, 0, np.shape(imm)[0]-1, save=png+'advance_clean')

OrderTraceAlg: idx:  1  status:  {'msg': 'same', 'cluster_id': 1, 'poly_fitting': {1: {'errors': 0.4961088520741995, 'coeffs': array([-2.67163632e-08,  3.10278671e-04, -1.19920733e+00,  1.54276349e+03,
        3.54600000e+03,  4.07900000e+03,  0.00000000e+00,  3.00000000e+00]), 'area': [3546, 4079, 0, 3]}}}
OrderTraceAlg: idx:  2  status:  {'msg': 'same', 'cluster_id': 2, 'poly_fitting': {2: {'errors': 2.5829773898579647, 'coeffs': array([-8.88372699e-11, -4.21526700e-09,  8.76771056e-03,  1.78252834e+00,
        0.00000000e+00,  4.07900000e+03,  0.00000000e+00,  3.60000000e+01]), 'area': [0, 4079, 0, 36]}}}
OrderTraceAlg: idx:  3  status:  {'msg': 'same', 'cluster_id': 3, 'poly_fitting': {3: {'errors': 3.361592269270169, 'coeffs': array([-4.12557474e-12, -6.29106664e-07,  1.00397418e-02,  1.99219223e+01,
        0.00000000e+00,  4.07900000e+03,  1.40000000e+01,  5.50000000e+01]), 'area': [0, 4079, 14, 55]}}}
OrderTraceAlg: idx:  4  status:  {'msg': 'same', 'cluster_id': 4, 'poly_fitti

OrderTraceAlg: idx:  40  status:  {'msg': 'same', 'cluster_id': 40, 'poly_fitting': {40: {'errors': 2.912217151868988, 'coeffs': array([-4.63302130e-11,  1.88023636e-07,  4.31916013e-03,  8.89835761e+02,
        0.00000000e+00,  4.07900000e+03,  8.85000000e+02,  9.12000000e+02]), 'area': [0, 4079, 885, 912]}}}
OrderTraceAlg: idx:  41  status:  {'msg': 'same', 'cluster_id': 41, 'poly_fitting': {41: {'errors': 1.560437417956987, 'coeffs': array([-3.36350277e-11,  1.40294020e-07,  4.36610962e-03,  9.04453024e+02,
        0.00000000e+00,  4.07900000e+03,  9.02000000e+02,  9.25000000e+02]), 'area': [0, 4079, 902, 925]}}}
OrderTraceAlg: idx:  42  status:  {'msg': 'same', 'cluster_id': 42, 'poly_fitting': {42: {'errors': 2.6613677466613472, 'coeffs': array([-3.54240825e-11,  2.04477739e-07,  3.95057401e-03,  9.53915470e+02,
        0.00000000e+00,  4.07900000e+03,  9.50000000e+02,  9.75000000e+02]), 'area': [0, 4079, 950, 975]}}}
OrderTraceAlg: idx:  43  status:  {'msg': 'same', 'cluster_id':

OrderTraceAlg: idx:  78  status:  {'msg': 'same', 'cluster_id': 78, 'poly_fitting': {78: {'errors': 2.6520686584557605, 'coeffs': array([ 3.33792425e-12,  5.97548314e-07, -1.50721398e-03,  1.95281173e+03,
        0.00000000e+00,  4.07900000e+03,  1.94800000e+03,  1.96100000e+03]), 'area': [0, 4079, 1948, 1961]}}}
OrderTraceAlg: idx:  79  status:  {'msg': 'same', 'cluster_id': 79, 'poly_fitting': {79: {'errors': 3.376537282278121, 'coeffs': array([-5.85316757e-11,  9.83889988e-07, -2.26333904e-03,  1.97208643e+03,
        0.00000000e+00,  4.07900000e+03,  1.96500000e+03,  1.98100000e+03]), 'area': [0, 4079, 1965, 1981]}}}
OrderTraceAlg: idx:  80  status:  {'msg': 'same', 'cluster_id': 80, 'poly_fitting': {80: {'errors': 2.9160395655374374, 'coeffs': array([-6.96289174e-11,  1.04033034e-06, -2.35269394e-03,  1.99098441e+03,
        0.00000000e+00,  4.07900000e+03,  1.98500000e+03,  1.99800000e+03]), 'area': [0, 4079, 1985, 1998]}}}
OrderTraceAlg: idx:  81  status:  {'msg': 'same', 'clust

OrderTraceAlg: idx:  114  status:  {'cluster_id': 114, 'poly_fitting': {}, 'cluster_added': [], 'msg': 'delete'}
OrderTraceAlg: idx:  115  status:  {'cluster_id': 115, 'poly_fitting': {}, 'cluster_added': [], 'msg': 'delete'}
OrderTraceAlg: idx:  116  status:  {'msg': 'same', 'cluster_id': 116, 'poly_fitting': {116: {'errors': 2.6997480476219318, 'coeffs': array([ 4.16042929e-12,  1.37806782e-06, -8.45352229e-03,  3.12203434e+03,
        0.00000000e+00,  4.07900000e+03,  3.10500000e+03,  3.12600000e+03]), 'area': [0, 4079, 3105, 3126]}}}
OrderTraceAlg: idx:  117  status:  {'msg': 'same', 'cluster_id': 117, 'poly_fitting': {117: {'errors': 3.4114356711781344, 'coeffs': array([-6.08691076e-11,  1.75804912e-06, -9.12394673e-03,  3.14146987e+03,
        0.00000000e+00,  4.07900000e+03,  3.12300000e+03,  3.14700000e+03]), 'area': [0, 4079, 3123, 3147]}}}
OrderTraceAlg: idx:  118  status:  {'msg': 'same', 'cluster_id': 118, 'poly_fitting': {118: {'errors': 2.961762828674279, 'coeffs': array(

OrderTraceAlg: idx:  150  status:  {'cluster_id': 150, 'poly_fitting': {}, 'cluster_added': [], 'msg': 'delete'}
OrderTraceAlg: idx:  151  status:  {'cluster_id': 151, 'poly_fitting': {}, 'cluster_added': [], 'msg': 'delete'}
OrderTraceAlg: idx:  152  status:  {'cluster_id': 152, 'poly_fitting': {}, 'cluster_added': [], 'msg': 'delete'}
OrderTraceAlg: idx:  153  status:  {'cluster_id': 153, 'poly_fitting': {}, 'cluster_added': [], 'msg': 'delete'}
OrderTraceAlg: idx:  154  status:  {'msg': 'same', 'cluster_id': 154, 'poly_fitting': {154: {'errors': 2.685406090547926, 'coeffs': array([-4.34247556e-12,  1.82844007e-06, -1.20377803e-02,  3.70896973e+03,
        0.00000000e+00,  4.07900000e+03,  3.68500000e+03,  3.71300000e+03]), 'area': [0, 4079, 3685, 3713]}}}
OrderTraceAlg: idx:  155  status:  {'msg': 'same', 'cluster_id': 155, 'poly_fitting': {155: {'errors': 3.4025656021322797, 'coeffs': array([-1.58680724e-11,  1.90441116e-06, -1.23049684e-02,  3.72827568e+03,
        0.00000000e+00,

/Users/cwang/Documents/KPF/KPF-Pipeline/venv/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


### save advanced cleaning result of step 4 to fits and info fits (optional)

In [ ]:
plot_img(make_cluster_fits(new_index, new_x, new_y, nx, ny), 0, ny-1)

In [ ]:
imm_advanced = make_cluster_fits(new_index, new_x, new_y, nx, ny, cluster_after_removal_fits)
make_cluster_info_csv(cluster_info_after_removal_csv, nx, ny, new_x, new_y, new_index)

#plot_img(imm_advanced, 0, np.shape(imm_advanced)[0]-1, save=png+'advance_clean2')

### reload advanced cleaning results from result of step 4 (optional)

In [ ]:
new_imm_after_removal, hdr = fits.getdata(cluster_after_removal_fits, header=True)
plot_img(new_imm_after_removal, 0, np.shape(new_imm_after_removal)[0]-1)
new_x, new_y, new_index = get_cluster_info_csv(cluster_info_after_removal_csv)

## 5. clean_clusters_on_borders() or clean_clusters_on_border() for top and bottom border 

In [22]:
x, y, index_b = order_t.clean_clusters_on_border(new_x, new_y, new_index, 0)
print(len(new_index))
print(len(index_b))

new_x, new_y, new_index = order_t.clean_clusters_on_border(x, y, index_b, ny-1)

print(len(index_b))
print(len(new_index))
imm = order_t.make_2d_data(new_index, new_x, new_y)   # show image  and make fits and info fits 
plot_img(imm, 0, np.shape(imm)[0]-1, save=png+'clean_border')

5223515
5220816
5220816
5220816
is bw image


/Users/cwang/Documents/KPF/KPF-Pipeline/venv/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


###  saving result of step 5 to fits and info fits (optional)

In [23]:
cluster_border_fits = output_img_dir +'/cluster_clean_border_info_' + png + '.fits'
cluster_info_border_csv = output_dir + '/cluster_clean_border_info_'+ png+'.csv'
imm_border = make_cluster_fits(new_index, new_x, new_y, nx, ny, cluster_border_fits)
make_cluster_info_csv(cluster_info_border_csv, nx, ny, new_x, new_y, new_index)
plot_img(imm_border, 0, ny-1)

there are 147 clusters in total in fits, ../../modules/order_trace/results/result_img/cluster_clean_border_info_KP.20220518.17215.78_GREEN_CCD.fits
is bw image


/Users/cwang/Documents/KPF/KPF-Pipeline/venv/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


### reload result of step 5 fits and info fits (optional)

In [ ]:
cluster_border_fits = output_img_dir +'/cluster_clean_border_info_' + png + '.fits'
cluster_info_border_csv = output_dir + '/cluster_clean_border_info_'+ png+'.csv'
imm_border, hdr = fits.getdata(cluster_border_fits, header=True)
plot_img(imm_border, 0, np.shape(imm_border)[0]-1)

new_x, new_y, new_index = get_cluster_info_csv(cluster_info_border_csv)

## plot order trace of step 5

In [ ]:
# prepare for step 6
imm_border = make_2D_data(new_index, new_x, new_y, nx, ny)
new_coeffs, errors = order_t.curve_fitting_on_all_clusters(new_index, new_x, new_y)
max_index = np.amax(new_index)

print(np.shape(new_coeffs))
print(max_index)
plot_poly_trace(imm_border, max_index, new_coeffs, nx, ny, save=png+'before_merge')

## 6. merge_cluster() -  Merging clusters

In [24]:
new_x, new_y, new_index = order_t.merge_clusters_and_clean(new_index, new_x, new_y)
imm = order_t.make_2d_data(new_index, new_x, new_y)   # show image  and make fits and info fits 
#plot_img(imm, 0, np.shape(imm)[0]-1, save=png+'merge')
max_index = np.amax(new_index)
new_coeffs, errors = order_t.curve_fitting_on_all_clusters(new_index, new_x, new_y)
plot_poly_trace(imm, max_index, new_coeffs, nx, ny, save=png+'after_merge')

OrderTraceAlg: merge time:  1
OrderTraceAlg: remove:  1  from:  1
OrderTraceAlg: merge time:  2
OrderTraceAlg: remove:  117  from:  117
OrderTraceAlg: merge time:  3
OrderTraceAlg: remove:  121  from:  121
OrderTraceAlg: merge time:  4
OrderTraceAlg: remove:  125  from:  125
OrderTraceAlg: merge time:  5
OrderTraceAlg: merge:  129 130  from:  129 130
OrderTraceAlg: merge time:  6
OrderTraceAlg: remove:  129  from:  129
OrderTraceAlg: merge time:  7
OrderTraceAlg: remove:  137  from:  137
OrderTraceAlg: merge time:  8
cluster  1  case  0
is bw image


/Users/cwang/Documents/KPF/KPF-Pipeline/venv/lib/python3.7/site-packages/ipykernel_launcher.py:59: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


### save merge results of step 6 to fits and info fits (optional)

In [ ]:
max_index = np.amax(new_index)
new_coeffs, errors = order_t.curve_fitting_on_all_clusters(new_index, new_x, new_y)
plot_poly_trace(imm, max_index, new_coeffs, nx, ny, save=png+'after_merge')

In [ ]:
# store after merge fits and fits info
imm = make_cluster_fits(new_index, new_x, new_y, nx, ny, cluster_merge_fitting)
make_cluster_info_csv(cluster_info_merge_fitting_csv, nx, ny, new_x, new_y, new_index)
plot_img(imm, 0, np.shape(imm)[0]-1)

### reload merge result of step 6  (optional)

In [ ]:
imm, hdr = fits.getdata(cluster_merge_fitting, header=True)
plot_img(imm, 0, np.shape(imm)[0]-1)

new_x, new_y, new_index = get_cluster_info_csv(cluster_info_merge_fitting_csv)

### plot order trace of result of step 6

In [ ]:
new_coeffs, errors = order_t.curve_fitting_on_all_clusters(new_index, new_x, new_y)
max_index = np.amax(new_index)    
plot_poly_trace(imm, max_index, new_coeffs, nx, ny, save=png+'trace')

## 7. find_all_cluster_widths() -- find top and bottom widths of the orders

In [ ]:
cluster_widths, cluster_coeffs =  order_t.find_all_cluster_widths(new_index, new_x, new_y, power_for_width_estimation=3)

In [ ]:
max_index = np.amax(new_index) 

df = order_t.write_cluster_info_to_dataframe(cluster_widths, cluster_coeffs)
df.to_csv('/Users/cwang/documents/KPF/KPF-Pipeline/test_results/kpf_04132022/output_trace/'+png+'.csv')
origin = [df.attrs['STARTCOL'], df.attrs['STARTROW']]
widths = [[w['bottom_edge'], w['top_edge']] for w in cluster_widths]
# plot_poly_trace(flat_data, max_index, cluster_coeffs, total_col, total_row, origin=origin, save=png+'width_edge')

In [ ]:
new_img = np.zeros((total_row, total_col), dtype='int')
if data_range is not None:
    new_img[data_range[2]:(data_range[3]+1), data_range[0]:(data_range[1]+1)] = imm[:, :]
else:
    new_img = imm[:, :]
plot_poly_trace(imm_spec, max_index, cluster_coeffs, total_col, total_row, origin=origin, save=png+'width_2')

### output to Pandas dataframe and then to csv  

In [ ]:
fits_data = ['assembled_KPF_220316_153200_L0.fits', 'socal_30s_bkgremoved.fits', 'KPF_220316_171800_bkgremoved.fits']
result_csv = '/Users/cwang/documents/KPF/KPF-Pipeline/test_results/kpf_new_data/output_trace/assembled_KPF_220316_153200_L0_GREEN_CCD.csv'
lev0_fits = '/Users/cwang/documents/KPF/KPF-Pipeline/test_data/kpf/220316/lev0/' + fits_data[1]

df = pd.read_csv(result_csv, header=0, index_col=0)
order_trace_data = np.array(df)
order_coeffs = np.flip(order_trace_data[:, 0:(power+1)], axis=1)
order_trace_data[:, 0:(power+1)] = order_coeffs
order_trace_data[:, (power+1):(power+3)] = order_trace_data[:, (power+3):(power+5)]

fits_header = fits.open(lev0_fits)
lev0_data = fits_header[0].data


In [ ]:
max_index = np.shape(order_trace_data)[0]
max_col = np.shape(order_trace_data)[1]
print(order_trace_data)
import pdb;pdb.set_trace()
order_trace_data = np.insert(order_trace_data, 0, np.zeros((1, max_col)), axis=0)
print(order_trace_data)
total_row, total_col = np.shape(lev0_data)
origin = [0, 0]
print(total_row, total_col)

plot_poly_trace(lev0_data, max_index, order_trace_data, total_col, total_row, origin=origin, save='trace_on_lev0', shift=0)